In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
     (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [57]:
import os
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report

ROOT = '/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup'
STEMS_PATH = os.path.join(ROOT, 'genres_stems')

GENRES = ["blues", "classical", "country", "disco",
          "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

In [58]:
jazz_path = os.path.join(STEMS_PATH, "jazz")

durations = []

for song in tqdm(os.listdir(jazz_path)):
    song_path = os.path.join(jazz_path, song)
    
    if os.path.isdir(song_path):
        for stem in os.listdir(song_path):
            file_path = os.path.join(song_path, stem)
            
            if os.path.exists(file_path):
                try:
                    y, sr = librosa.load(file_path, sr=None)
                    durations.append(len(y) / sr)
                except:
                    continue

print("Answer Q1 (Mean Duration Jazz):", np.mean(durations))

100%|██████████| 100/100 [00:38<00:00,  2.62it/s]

Answer Q1 (Mean Duration Jazz): 30.032979591836728


In [59]:
sample_rates = set()

for g in GENRES:
    genre_path = os.path.join(STEMS_PATH, g)
    
    for song in os.listdir(genre_path):
        song_path = os.path.join(genre_path, song)
        
        for stem in os.listdir(song_path):
            file_path = os.path.join(song_path, stem)
            
            if os.path.exists(file_path):
                try:
                    y, sr = librosa.load(file_path, sr=None)
                    sample_rates.add(sr)
                except:
                    continue

print("Answer Q2 (Unique Sample Rates):", sorted(list(sample_rates)))

Answer Q2 (Unique Sample Rates): [44100]


In [60]:
corrupted_count = 0

for g in GENRES:
    genre_path = os.path.join(STEMS_PATH, g)
    
    for song in os.listdir(genre_path):
        song_path = os.path.join(genre_path, song)
        
        for stem in os.listdir(song_path):
            file_path = os.path.join(song_path, stem)
            
            if os.path.exists(file_path):
                if os.path.getsize(file_path) == 0:
                    corrupted_count += 1

print("Answer Q3 (Corrupted Files):", corrupted_count)

Answer Q3 (Corrupted Files): 0


In [61]:
peak_db_values = []

for g in GENRES:
    genre_path = os.path.join(STEMS_PATH, g)
    
    for song in os.listdir(genre_path):
        song_path = os.path.join(genre_path, song)
        
        vocal_path = os.path.join(song_path, "vocals.wav")
        
        if os.path.exists(vocal_path):
            try:
                y, sr = librosa.load(vocal_path, sr=None)
                
                if len(y) > 0:
                    peak = np.max(np.abs(y))
                    peak_db = 20 * np.log10(peak + 1e-10)
                    peak_db_values.append(peak_db)
                    
            except:
                continue

print("Answer Q4 (Avg Peak dB Vocals):", np.mean(peak_db_values))

Answer Q4 (Avg Peak dB Vocals): -12.494921


In [62]:
blues_path = os.path.join(STEMS_PATH, "blues")

centroids = []

for song in os.listdir(blues_path):
    song_path = os.path.join(blues_path, song)
    
    for fname in ['other.wav', 'others.wav']:
        file_path = os.path.join(song_path, fname)
        
        if os.path.exists(file_path):
            try:
                y, sr = librosa.load(file_path, sr=22050)
                
                if len(y) > 0:
                    spec_cent = np.mean(
                        librosa.feature.spectral_centroid(y=y, sr=sr)
                    )
                    centroids.append(spec_cent)
            except:
                continue

print("Answer Q5 (Mean Spectral Centroid Blues):", np.mean(centroids))

Answer Q5 (Mean Spectral Centroid Blues): 1597.8387497380847


In [63]:
genre_centroids = {}

for g in GENRES:
    centroids = []
    genre_path = os.path.join(STEMS_PATH, g)
    
    for song in os.listdir(genre_path):
        song_path = os.path.join(genre_path, song)
        
        for fname in ['other.wav', 'others.wav']:
            file_path = os.path.join(song_path, fname)
            
            if os.path.exists(file_path):
                try:
                    y, sr = librosa.load(file_path, sr=22050)
                    
                    if len(y) > 0:
                        spec_cent = np.mean(
                            librosa.feature.spectral_centroid(y=y, sr=sr)
                        )
                        centroids.append(spec_cent)
                except:
                    continue
    
    if len(centroids) > 0:
        genre_centroids[g] = np.mean(centroids)

print("All Genre Means:", genre_centroids)
print("Answer Q6 (Highest Centroid Genre):",
      max(genre_centroids, key=genre_centroids.get))

All Genre Means: {'blues': np.float64(1597.8387497380847), 'classical': np.float64(1357.8935316163452), 'country': np.float64(1363.486739099068), 'disco': np.float64(1798.606654557341), 'hiphop': np.float64(2429.2962754907903), 'jazz': np.float64(1554.8343991374256), 'metal': np.float64(2418.801885494158), 'pop': np.float64(1517.4679898262727), 'reggae': np.float64(1761.585430812623), 'rock': np.float64(1547.635900011739)}
Answer Q6 (Highest Centroid Genre): hiphop


In [64]:
silence_count = 0

for g in GENRES:
    genre_path = os.path.join(STEMS_PATH, g)
    
    for song in os.listdir(genre_path):
        song_path = os.path.join(genre_path, song)
        
        for stem in os.listdir(song_path):
            file_path = os.path.join(song_path, stem)
            
            if os.path.exists(file_path):
                try:
                    y, sr = librosa.load(file_path, sr=None)
                    
                    first_half_sec = y[:int(0.5 * sr)]
                    
                    if np.max(np.abs(first_half_sec)) < 1e-4:
                        silence_count += 1
                        
                except:
                    continue

print("Answer Q7 (Silence Count):", silence_count)

Answer Q7 (Silence Count): 333


In [65]:
def extract_features_safe(song_path):
    
    # auto detect stem name
    possible_files = ['other.wav', 'others.wav']
    file_path = None
    
    for fname in possible_files:
        temp_path = os.path.join(song_path, fname)
        if os.path.exists(temp_path):
            file_path = temp_path
            break
    
    if file_path is None:
        return None
    
    try:
        y, sr = librosa.load(file_path, sr=22050, duration=10)
        
        if len(y) == 0:
            return None
        
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        
        return [float(tempo), spec_cent, zcr, rolloff]
    
    except:
        return None

In [66]:
data = []

for g in GENRES:
    gp = os.path.join(STEMS_PATH, g)
    songs = [s for s in os.listdir(gp) if os.path.isdir(os.path.join(gp, s))]
    
    for s in songs[:50]:   # speed ke liye
        data.append({'path': os.path.join(gp, s), 'genre': g})

df = pd.DataFrame(data)

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['genre'],
    random_state=42
)

In [67]:
X_train = []
y_train = []

for path, genre in zip(train_df['path'], train_df['genre']):
    features = extract_features_safe(path)
    if features is not None:
        X_train.append(features)
        y_train.append(genre)

X_val = []
y_val = []

for path, genre in zip(val_df['path'], val_df['genre']):
    features = extract_features_safe(path)
    if features is not None:
        X_val.append(features)
        y_val.append(genre)

X_train = np.array(X_train)
X_val = np.array(X_val)

print("Train samples:", len(X_train))
print("Validation samples:", len(X_val))

/tmp/ipykernel_55/3414979543.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return [float(tempo), spec_cent, zcr, rolloff]


Train samples: 400
Validation samples: 100


In [68]:
clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [69]:
y_pred = clf.predict(X_val)

macro_f1 = f1_score(y_val, y_pred, average='macro')
print("Answer Q8 (Validation Macro F1):", macro_f1)

Answer Q8 (Validation Macro F1): 0.15230042016806722


In [70]:
cr = classification_report(y_val, y_pred, output_dict=True)

print("Answer Q9 (Precision hiphop):", cr['hiphop']['precision'])

Answer Q9 (Precision hiphop): 0.25


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [71]:
print("Answer Q10 (Recall pop):", cr['pop']['recall'])

Answer Q10 (Recall pop): 0.2


In [72]:
accuracy = np.mean(y_pred == y_val)
print("Answer Q11 (Accuracy):", accuracy)

Answer Q11 (Accuracy): 0.19


In [73]:
cm = confusion_matrix(y_val, y_pred, labels=GENRES)

tp_dict = {}

for i, genre in enumerate(GENRES):
    TP = cm[i, i]
    tp_dict[genre] = TP

print("Answer Q12 (Highest TP Genre):", max(tp_dict, key=tp_dict.get))

Answer Q12 (Highest TP Genre): metal


In [74]:
fn_dict = {}

for i, genre in enumerate(GENRES):
    TP = cm[i, i]
    FN = np.sum(cm[i, :]) - TP
    fn_dict[genre] = FN

print("Answer Q13 (Lowest FN Genre):", min(fn_dict, key=fn_dict.get))

Answer Q13 (Lowest FN Genre): metal
